In [0]:
import cv2 as cv2
from PIL import Image
import numpy as np
import os



def read_image_from_path(image_path):
    return Image.open(image_path)



def resize_image_keeping_aspect_ratio(image, new_width):
#     color_image = Image.open(image_path)
    # img = Image.open('somepic.jpg')
    wpercent = (new_width / float(image.size[0]))
    hsize = int((float(image.size[1]) * float(wpercent)))
    color_image = image.resize((new_width, hsize), Image.ANTIALIAS)
#     color_image.show()
    print(np.shape(color_image))
    return np.array(color_image)



def convert_rgb_to_bw(color_image):
    color_image = Image.fromarray(color_image)
    gray_image = color_image.convert('L')
    bw_image = gray_image.point(lambda x: 0 if x<128 else 255, '1')
#     bw_image.show()
    return np.array(bw_image)
    
    
    


def image_dilate_and_gaussian_smoothing(image_path, image_name, kernel_size = 5):
#     kernel_size = 5
#     image_path = '/Users/vijay/Downloads/Ishu.jpg'
#     print('started')
#     color_image = read_image_from_path(image_path)
    # bw_image    = convert_rgb_to_bw(image_path)
#     color_image = resize_image_keeping_aspect_ratio(color_image, 256)
    color_image = cv2.resize(cv2.imread(image_path), (256, 256))
#     bw_image_rs    = convert_rgb_to_bw(color_image)

    kernel = np.ones((5, 5), np.uint8)
    pad_img = np.pad(color_image, ((2, 2), (2, 2), (0, 2)), mode = 'reflect')


#     bw_image_ = cv2.cvtColor(bw_image_rs.astype(np.uint8), cv2.COLOR_GRAY2BGR)
    bw_image_ = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(bw_image_,100,200)
    # Image.fromarray(edges).show()
    dilate_image = cv2.dilate(edges, kernel)
    gauss_kernel = cv2.getGaussianKernel(5, 0)
    gauss_kernel = gauss_kernel * gauss_kernel.transpose(1, 0)


    gauss_kernel = np.array(gauss_kernel)
    color_image_ = np.array(color_image)
    idx = np.where(dilate_image != 0)
    for i in range(np.sum(dilate_image != 0)):

        color_image_[idx[0][i], idx[1][i], 0] = np.sum(
            np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 0], gauss_kernel))
        color_image_[idx[0][i], idx[1][i], 1] = np.sum(
            np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 1], gauss_kernel))
        color_image_[idx[0][i], idx[1][i], 2] = np.sum(
            np.multiply(pad_img[idx[0][i]:idx[0][i] + kernel_size, idx[1][i]:idx[1][i] + kernel_size, 2], gauss_kernel))
#     Image.fromarray(color_image_).show()
    Image.fromarray(color_image_).save('/content/drive/My Drive/CartoonGAN/train_Y_smoothed/' + image_name)

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
images_path = '/content/drive/My Drive/CartoonGAN/train_Y/train_Y/'
images_names = os.listdir(images_path)

for image_name in images_names:
    if '.DS_Store' not in image_name:
        image_dilate_and_gaussian_smoothing(images_path + image_name, image_name)

In [8]:
names = os.listdir('/content/drive/My Drive/CartoonGAN/train_Y_smoothed/')
print(len(names))

5001
